In [2]:
import os
import pandas as pd

In [4]:
import os
import pandas as pd

mimic_iv_cxr_parent = "/cis/home/charr165/Documents/physionet.org/files/mimic-cxr-jpg/2.0.0"

mm_dir = "/cis/home/charr165/Documents/multimodal"
preprocessing_dir = os.path.join(mm_dir, "preprocessing")
f_path = os.path.join(preprocessing_dir, "cxr_embeddings.pkl")

In [ ]:
cxr_embeddings_df = pd.read_pickle(f_path)
cxr_embeddings_df['cxrtime'] = pd.to_datetime(cxr_embeddings_df['cxrtime'])

mimic_iv_path = "/cis/home/charr165/Documents/physionet.org/mimiciv/2.2"
icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv"), low_memory=False)
admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'])
admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'])

In [ ]:
from tqdm import tqdm

def calc_time_delta_hrs(icu_intime, charttime):
    return (charttime - icu_intime).total_seconds() / 3600

cxr_embeddings_df['hadm_id'] = None
cxr_embeddings_df['stay_id'] = None
cxr_embeddings_df['icu_time_delta'] = None
cxr_embeddings_df['hosp_time_delta'] = None

for index, row in tqdm(cxr_embeddings_df.iterrows(), total=cxr_embeddings_df.shape[0]):
    curr_pts_icustays = icustays_df[icustays_df['subject_id'] == row['subject_id']]
    
    for icu_index, icu_row in curr_pts_icustays.iterrows():
        if icu_row['intime'] <= row['cxrtime'] <= icu_row['outtime']:
            cxr_embeddings_df.loc[index, 'stay_id'] = icu_row['stay_id']
            cxr_embeddings_df.loc[index, 'icu_time_delta'] = calc_time_delta_hrs(icu_row['intime'], row['cxrtime'])
    
    curr_pts_admissions = admissions_df[admissions_df['subject_id'] == row['subject_id']]

    for hosp_index, hosp_row in curr_pts_admissions.iterrows():
        if hosp_row['admittime'] <= row['cxrtime'] <= hosp_row['dischtime']:
            cxr_embeddings_df.loc[index, 'hadm_id'] = hosp_row['hadm_id']
            cxr_embeddings_df.loc[index, 'hosp_time_delta'] = calc_time_delta_hrs(hosp_row['admittime'], row['cxrtime'])

In [ ]:
cxr_embeddings_df.to_pickle(os.path.join(preprocessing_dir, "cxr_embeddings_stay.pkl"))

In [7]:
cxr_embeddings_df['stay_id'].isna().mean()

0.6118373949245578

In [5]:
cxr_embeddings_df = pd.read_pickle(os.path.join(preprocessing_dir, "cxr_embeddings_stay.pkl"))